In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import requests
import wikipedia
import re
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import urlopen
from scrapy import selector
import datetime as dt
import pickle
from skimage import io
from IPython.display import clear_output
import pdb
plt.style.use('ggplot')
%matplotlib inline

# Functions

In [78]:
def get_squad(batting_sheet):
    """Gets team members, captain and keeper
    """
    team = []
    
    # batsmen who batted
    batsmen = batting_sheet.find_all('div', class_ = "wrap batsmen")
    for bat in batsmen:
        player = bat.find('div', class_ = "cell batsmen").text
        if player.find('(c)')>-1:
            captain = player.replace('(c)','').replace('†','').strip().lower()
        if player.find('†')>-1:
            keeper = player.replace('†','').replace('(c)','').strip().lower()
        player = player.replace('†','').replace('(c)','').strip()
        team.append(player.lower())
    did_not_bat = batting_sheet.find('div', class_ = "wrap dnb")
    
    
    # Now batsmen who did not bat
    for dnb in did_not_bat.find_all('a'):
        dnb = dnb.text.lower()
        if dnb.find('(c)')>-1:
            captain = dnb.replace('(c)','').replace('†','').strip().lower()
        if dnb.find('†')>-1:
            keeper = dnb.replace('†','').replace('(c)','').strip().lower()
        team.append(dnb.replace('†','').replace('(c)','').strip().lower())
    return team, captain, keeper


def get_full_name(name, team):
    """Accepts last name and returns players name
    """
    
    if len(team)==0:
        return name
    name = name.lower()
    name = name.replace('†','').replace('(c)','')
    for member in team:
        if member.lower().find(name) > -1:
            return member
        
def get_innings(innings, teams):
    """Returns batting and bowling innings
    """
    
    for i,inning in enumerate(innings):
        for team in teams:
            if inning.find(team)>-1:
                innings[i] = team
    
    return innings
                

        
def get_toss_outcome(toss_string):
    pass

In [220]:

outs = ['b', 'c', 'lbw', 'st', 'hw', 'ro', 'hand', 'obs']

def how_out(dismissal, team):
    """Returns triplet tuple as (how, fielder, bowler)
    """
    
    bowler,fielder = '',''
    dismissal = dismissal.lower().strip()
    print(dismissal)
    if len(dismissal)==0:
        return '','',''
    
    
    if dismissal == 'not out':
        return 'not out', '', ''
    
    if dismissal.find('retired ')>-1:
        return 'retired hurt', '', ''
    
    if dismissal.find('absent') > -1 and dismissal.find('hurt') > -1:
        return 'absent hurt', '', ''
    
    # bowled
    if re.match(r'b\s.+', dismissal)!= None and dismissal == re.match(r'b\s.+', dismissal)[0]:
        how = 'bowled'
        bowler = re.findall(r"b\s(.+)", dismissal)[0]
        fielder = ''
        return how, bowler, fielder
        
    # if substitute
    if dismissal.find('(sub)')>-1:
        dismissal.replace('(sub)','')
        fielder = ['substitute']
    elif dismissal.find(' sub ')>-1:
        dismissal.replace(' sub ','')
        dismissal.replace('(','')
        dismissal.replace(')','')
        fielder = ['substitute']
    else:
        fielder = re.findall(r"\s([†\w\s'-]+)\sb", dismissal)
    
    bowler = re.findall(r"\sb\s([\w\s'-]+)", dismissal)
    if len(bowler)>0:
        bowler = bowler[0]
        
    if len(fielder)>0:
        fielder = fielder[0]
        fielder = fielder.replace('†','')
         
    
    
    # fielder = re.findall(r"\s([\w\s'-]+)\sb\s", dismissal)
    # print(bowler, fielder)
    
    ## No fielder involved
    # run out
    if dismissal.find('run out')>-1:
        how = 'run out'
        fielder = ''
        bowler = ''
        return how, fielder, bowler

    # handled
    if dismissal.find('handled')>-1:
        how = 'handled the ball'
        fielder = ''
        bowler = ''
        return how, fielder, bowler

    # obstructing the fieldsman
    if dismissal.find('obstruct') > -1:
        how = 'obstruct'
        fielder = ''
        blower = ''
        return how, fielder, bowler
    
        
    # caught and bowled
    if dismissal.find('&') > -1:
        how = 'caught'
        bowler = get_full_name(bowler, team)
        # fielder = bowler
        return how, bowler, bowler
    
    if len(fielder) == 0:
        keeper = re.findall(r"st\s†(.+)\sb", dismissal)
        if len(keeper)>0:
            keeper = keeper[0]
    
    # bowled
    if dismissal == 'b ' + bowler:
        how = 'bowled'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # caught
    if len(fielder) > 0:
        how = 'caught'
        fielder = get_full_name(fielder, team)
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # lbw
    if dismissal.find('lbw')>-1 or dismissal.find('leg before')>-1:
        how = 'lbw'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # hit wicket
    if dismissal.find('hit wicket')>-1:
        how = 'hit wicket'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler

    
    # Stumped
    if dismissal.find('st †') > -1:
        how = 'stumped'
        fielder = get_full_name(keeper, team)
        bowler = get_full_name(bowler, team)
        return 'stumped',fielder, bowler
    
    


In [65]:
match[1996]

,test_index,team_1,team_2,winner,margin,ground,match_date,scorecard_link
0,test # 1321,south africa,england,south africa,10 wickets,cape town,"jan 2-4, 1996",http://stats.espncricinfo.com//ci/engine/match...
1,test # 1322,new zealand,zimbabwe,drawn,,hamilton,"jan 13-17, 1996",http://stats.espncricinfo.com//ci/engine/match...
2,test # 1323,new zealand,zimbabwe,drawn,,auckland,"jan 20-24, 1996",http://stats.espncricinfo.com//ci/engine/match...
3,test # 1324,australia,sri lanka,australia,148 runs,adelaide,"jan 25-29, 1996",http://stats.espncricinfo.com//ci/engine/match...
4,test # 1325,west indies,new zealand,west indies,10 wickets,bridgetown,"apr 19-23, 1996",http://stats.espncricinfo.com//ci/engine/match...
5,test # 1326,west indies,new zealand,drawn,,st john's,"apr 27-may 2, 1996",http://stats.espncricinfo.com//ci/engine/match...
6,test # 1327,england,india,england,8 wickets,birmingham,"jun 6-9, 1996",http://stats.espncricinfo.com//ci/engine/match...
7,test # 1328,england,india,drawn,,lord's,"jun 20-24, 1996",http://stats.espncricinfo.com//ci/engine/match...
8,test # 1329,england,india,drawn,,nottingham,"jul 4-9, 1996",http://stats.espncricinfo.com//ci/engine/match...
9,test # 1330,england,pakistan,pakistan,164 runs,lord's,"jul 25-29, 1996",http://stats.espncricinfo.com//ci/engine/match...


In [223]:
%%time
def get_scoresheet(matchID, year, link, team_1, team_2, winner, match_format):
    """Get all information about a single match
    MATCH_DETAILS: Stadium, toss, umpires, etc
    TEAM_INFO: Teams, batting team, bowling team, squad, captyain, keeper
    BATTING_INFO: Batting scorecards of all innings
    BOWLING_INFO: Bowling Scorecards of all innings
    """


    # Initialize dictionary to be returned
    all_info = {}
    

    # Define opponents
    team_names = [team_1, team_2]
    opponents = {}
    opponents[team_1] = team_2
    opponents[team_2] = team_1

    # Send get request and get HTML
    r = requests.get(link)
    soup = BeautifulSoup(r.content,'lxml')

    # Number of score sheets = number of innings
    # Each scoresheet contains both batting and bowling information
    scoresheets = soup.find_all('article', class_ = "sub-module scorecard")
    match_details = soup.find('div', class_ = "match-detail-container")

    ##############################################################################################
    ############################# .   Get MATCH DETAILS   ############################################
    ##############################################################################################


    MATCH_DETAILS = {}
    MATCH_DETAILS['stadium'] = match_details.find('div', class_ = "stadium-details").text.lower().strip()
    for detail in match_details.find_all('div', class_ = "match-detail--item"):
        left = detail.find('div', class_ = "match-detail--left").text.lower().strip()
        MATCH_DETAILS[left] = []
        for r in detail.find('div', class_ = "match-detail--right").find_all('span'):
            MATCH_DETAILS[left].append(r.text.lower().strip())
    MATCH_DETAILS
    toss = re.findall(r'(.+),',MATCH_DETAILS['toss'][0])[0].strip()
    stadium = MATCH_DETAILS['stadium']


    ##############################################################################################
    ############################      Get TEAM INFO    ###########################################
    ##############################################################################################


    num_innings = len(scoresheets)

    bat_innings = []
    ball_innings = []

    squad = {}
    squad[team_1] = []
    squad[team_2] = []

    captain = {}
    captain[team_1] = []
    captain[team_2] = []

    keeper = {}
    keeper[team_1] = []
    keeper[team_2] = []

    # Get innings Info, team, captain, keeper
    for i,sheet in enumerate(scoresheets):


        if match_format == 'odi':
            my_str = sheet.find('div', class_ = "accordion-header").text.lower().strip()
            this_innings = re.findall(r'(.+)\sinnings', my_str)[0]
        elif match_format == 'test':
            my_str = sheet.find('div', class_ = "accordion-header").text.lower().strip()
            this_innings = re.findall(r'(.+)\s\d', my_str)[0]

        bat_innings.append(this_innings)

    ball_innings = [opponents[x] for x in bat_innings]

    # Get squad for team batting first
    team_batting_first = bat_innings[0]
    batting_sheet = scoresheets[0].find("div", class_="scorecard-section batsmen")
    squad[team_batting_first], captain[team_batting_first], keeper[team_batting_first] = get_squad(batting_sheet)

    # Get squad for team batting second
    team_batting_second = bat_innings[1]
    batting_sheet = scoresheets[1].find("div", class_="scorecard-section batsmen")
    squad[team_batting_second], captain[team_batting_second], keeper[team_batting_second] = get_squad(batting_sheet)


    TEAM_INFO = {}
    TEAM_INFO['num_innings'] = num_innings

    TEAM_INFO['bat_innings'] = bat_innings
    TEAM_INFO['ball_innings'] = ball_innings

    TEAM_INFO['team_batting_first'] = team_batting_first
    TEAM_INFO['team_batting_second'] = team_batting_second

    TEAM_INFO['squad'] = squad
    TEAM_INFO['captain'] = captain
    TEAM_INFO['keeper'] = keeper




    #############################################################################################
    ####################    Get Scoresheets/ Batting and Bowling        #########################
    #############################################################################################


    # Now get batting and bowling info
    BATTING = {}
    BOWLING = {}
    for s,sheet in enumerate(scoresheets):

        # batting_sheet = sheet.find("div", class_="scorecard-section batsmen")
        # bowling_sheet = sheet.find("div", class_ = "scorecard-section bowling")
        inning = 'inning_' + str(s+1)
        print("In:", inning)

        batting_team = bat_innings[s]
        bowling_team = ball_innings[s]

        #####################################################
        ############### Batting Info  #######################
        #####################################################


        batting_sheet = sheet.find("div", class_="scorecard-section batsmen")

        ## DEFINE HEADERS
        # get all the header.
        # Some games have SR and balls, some not, so important to get it on a game-by-game basis
        all_headers = []
        for header in batting_sheet.find('div', class_ = "wrap header").find_all('div'):
            all_headers.append(header.text.lower())

        # The commentary header is usually emptty: just add commentary
        if len(all_headers[1])==0:
            all_headers[1] = 'commentary'

        # Create dataframe
        all_headers =  all_headers + ['how', 'fielder', 'bowler'] + ['matchID', 'year', 'team', 'innings', 'toss', 'result', 'stadium']
        score_card_batting = pd.DataFrame(columns = all_headers)

        # Iterate over batesmen
        all_batsmen  = batting_sheet.find_all('div', class_ = "wrap batsmen")
        for i, batsman in enumerate(all_batsmen):

            # iterate over cells
            for j,info in enumerate(batsman.find_all('div')):
                # print(j,info.text, type(info.text))

                if j == 0: #name
                    player = info.text.lower().replace('(c)','').replace('†','').strip()
                    score_card_batting.loc[i, all_headers[j]] = player
                    continue
                if j == 1: #commentary
                    # print(info.text.lower().strip())
                    how,fielder,bowler = how_out(info.text.lower().strip(), squad[bowling_team])


                score_card_batting.loc[i, 'how'] = how
                score_card_batting.loc[i, 'fielder'] = fielder
                score_card_batting.loc[i, 'bowler'] = bowler
                score_card_batting.loc[i, all_headers[j]] = info.text.lower().strip()

        did_not_bat = set(squad[batting_team]) - set(score_card_batting.batsmen.values)
        #print(did_not_bat)

        l = len(score_card_batting)
        for dnb in did_not_bat:
            score_card_batting.loc[l, all_headers[0]] = dnb
            l = l+1

        # Now get extras
        extras = sheet.find('div', class_ = "wrap extras").find_all('div')[1].text
        l = len(score_card_batting)
        score_card_batting.loc[l, all_headers[0]] = 'extras'
        score_card_batting.loc[l, all_headers[1]] =  extras

        # Total
        total = sheet.find('div', class_ = "wrap total").find_all('div')[1].text
        l = len(score_card_batting)
        score_card_batting.loc[l, all_headers[0]] = 'total'
        score_card_batting.loc[l, all_headers[1]] =  total


        # Now fill match-cells: common to all batsmen
        score_card_batting.loc[:, 'matchID'] = matchID
        score_card_batting.loc[:, 'year'] = year
        score_card_batting.loc[:, 'innings'] = s+1
        score_card_batting.loc[:, 'team'] = batting_team

        score_card_batting.loc[:, 'stadium'] = stadium
        score_card_batting.loc[:, 'toss'] = toss
        score_card_batting.loc[:, 'result'] = winner

        BATTING[inning] = score_card_batting.fillna(0).replace('-','0')

        #####################################################
        ############### Bowling Info  #######################
        #####################################################


        bowling_sheet = sheet.find("div", class_ = "scorecard-section bowling")
        bowling_headers = []
        for header in bowling_sheet.find('thead').find_all('th'):
            bowling_headers.append(header.text.lower().strip())

        bowling_headers = bowling_headers +  ['matchID', 'year', 'team', 'innings']
        score_card_bowling = pd.DataFrame(columns = bowling_headers)  

        # bowlers who bowled
        for b,bowler in enumerate(bowling_sheet.find('tbody').find_all('tr')): 
            for c,cell in enumerate(bowler.find_all('td')):
                score_card_bowling.loc[b,bowling_headers[c]]= cell.text.lower().strip()
        # bowlers who did not bowl
        dnb = set(squad[bowling_team]) - set(score_card_bowling[bowling_headers[0]].values)
        l = len(score_card_bowling)
        for d in dnb:
            score_card_bowling.loc[l,bowling_headers[0]] = d
            l = l+1

        # Now fill match-cells: common to all bowlers
        score_card_bowling.loc[:, 'matchID'] = matchID
        score_card_bowling.loc[:, 'year'] = year
        score_card_bowling.loc[:, 'innings'] = s+1
        score_card_bowling.loc[:, 'team'] = bowling_team

        score_card_bowling.loc[:, 'stadium'] = stadium
        score_card_bowling.loc[:, 'toss'] = toss
        score_card_bowling.loc[:, 'result'] = winner

        BOWLING[inning] = score_card_bowling.fillna(0).replace('-','0')



    ## SAVE EVERYTHING IN 1 DICTIONARY
    all_info = {}
    all_info['link'] = link
    all_info['format'] = match_format
    all_info['team_info'] = TEAM_INFO
    all_info['match_details'] = MATCH_DETAILS
    all_info['batting'] = BATTING
    all_info['bowling'] = BOWLING

    return all_info

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.06 µs


In [224]:
tests.keys()
tests[1932]

,team_1,team_2,winner,margin,ground,match_date,scorecard_link
test_index,,,,,,,
test # 214,australia,south africa,australia,169 runs,melbourne,"dec 31, 1931 - jan 6, 1932",http://stats.espncricinfo.com//ci/engine/match...
test # 215,australia,south africa,australia,10 wickets,adelaide,"jan 29-feb 2, 1932",http://stats.espncricinfo.com//ci/engine/match...
test # 216,australia,south africa,australia,inns & 72 runs,melbourne,"feb 12-15, 1932",http://stats.espncricinfo.com//ci/engine/match...
test # 217,new zealand,south africa,south africa,inns & 12 runs,christchurch,"feb 27-mar 1, 1932",http://stats.espncricinfo.com//ci/engine/match...
test # 218,new zealand,south africa,south africa,8 wickets,wellington,"mar 4-7, 1932",http://stats.espncricinfo.com//ci/engine/match...
test # 219,england,india,england,158 runs,lord's,"jun 25-28, 1932",http://stats.espncricinfo.com//ci/engine/match...
test # 220,australia,england,england,10 wickets,sydney,"dec 2-7, 1932",http://stats.espncricinfo.com//ci/engine/match...
test # 221,australia,england,australia,111 runs,melbourne,"dec 30, 1932 - jan 3, 1933",http://stats.espncricinfo.com//ci/engine/match...


# Get Test Match Data

In [225]:
tests = pickle.load(open('test_list', "rb" ))
info = {}

In [241]:
skip_tests = ['test # 71', 'test # 156', 'test # 163', 'test # 211', 'test # 650', 'test # 1076',
             'test # 1179', 'test # 1226','test # 1309','test # 1312', 'test # 1387', 'test # 1396',
             'test # 1483', 'test # 1502', 'test # 1793', 'test # 1907', 'test # 1982', 'test # 2174',
             'test # 2218']
match_format = 'test'
for year in tests.keys():
    clear_output()
#     if year != 2019:
#         continue
    table = tests[year]
    for row in table.iterrows():
        clear_output()
        matchID = row[0]
        team1 = row[1].team_1
        team2 = row[1].team_2
        winner = row[1].winner
        link = row[1].scorecard_link
        ground = row[1].ground
        
        print(matchID)
        print(link)
        
        if matchID in skip_tests:
            continue
        if matchID not in info:
            info[matchID] = get_scoresheet(matchID, year, link, team1, team2, winner, match_format)


test # 2382
http://stats.espncricinfo.com//ci/engine/match/1185307.html
In: inning_1
c van der dussen b philander
c †de kock b hendricks
c van der dussen b paterson
c †de kock b nortje
c van der dussen b nortje
b nortje
c elgar b philander
c †de kock b nortje
c du plessis b nortje
not out
c malan b paterson
In: inning_2
c woakes b stokes
c †buttler b wood
c stokes b curran
lbw b woakes
b wood
c stokes b wood
c denly b wood
c broad b woakes
c crawley b stokes
not out
c †buttler b wood
In: inning_3
c †de kock b pretorius
c malan b hendricks
b paterson
c du plessis b hendricks
c van der dussen b hendricks
c †de kock b nortje
c †de kock b nortje
c paterson b hendricks
c †de kock b hendricks
b pretorius
not out
In: inning_4
c stokes b woakes
c & b stokes
c broad b wood
b stokes
c woakes b wood
c †buttler b broad
c curran b broad
c †buttler b wood
c †buttler b wood
run out (sibley/†buttler)
not out


In [244]:
info['missed_matches'] = skip_tests
pickle.dump(info,open('test_info_new', "wb" ))

In [243]:
info['test # 1388']

{'link': 'http://stats.espncricinfo.com//ci/engine/match/63771.html',
 'format': 'test',
 'team_info': {'num_innings': 4,
  'bat_innings': ['australia', 'new zealand', 'australia', 'new zealand'],
  'ball_innings': ['new zealand', 'australia', 'new zealand', 'australia'],
  'team_batting_first': 'australia',
  'team_batting_second': 'new zealand',
  'squad': {'australia': ['mtg elliott',
    'ma taylor',
    'gs blewett',
    'me waugh',
    'sr waugh',
    'rt ponting',
    'ia healy',
    'pr reiffel',
    'sk warne',
    'ms kasprowicz',
    'sh cook'],
   'new zealand': ['ba young',
    'mj horne',
    'ac parore',
    'sp fleming',
    'rg twose',
    'cd mcmillan',
    'nj astle',
    'cl cairns',
    'dl vettori,',
    'sb doull,',
    "sb o'connor"]},
  'captain': {'australia': 'ma taylor', 'new zealand': 'sp fleming'},
  'keeper': {'australia': 'ia healy', 'new zealand': 'ac parore'}},
 'match_details': {'stadium': 'bellerive oval, hobart',
  'series': ['new zealand tour of au